In [5]:
import torch
from vision.projector import load_vision_projector
from vision.feature_select import feature_select
from image_handling.padding import resize_with_padding, load_images
from vision.learned_encoding import load_learned_encoding
import requests
from PIL import Image
from io import BytesIO

In [6]:

from transformers import BitsAndBytesConfig , CLIPVisionModel, CLIPImageProcessor, LlamaForCausalLM, AutoTokenizer , LlamaTokenizer , AutoModelForCausalLM
  
quantization_config = BitsAndBytesConfig(load_in_8bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

def load_vision_model(pretrained_model, device = "cuda", cache_dir = None ):
    vision_model = CLIPVisionModel.from_pretrained(pretrained_model, cache_dir=cache_dir)
    image_processor = CLIPImageProcessor.from_pretrained(pretrained_model,  cache_dir=cache_dir)
    return vision_model , image_processor;

def load_llm(pretrained_model, device = "cuda", cache_dir = None ):
    llm = AutoModelForCausalLM.from_pretrained(pretrained_model,  cache_dir=cache_dir, quantization_config = quantization_config)
    llm.share_memory()
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model, cache_dir=cache_dir )
    return llm, tokenizer

RuntimeError: Failed to import transformers.models.clip.modeling_clip because of the following error (look up to see its traceback):
Failed to import transformers.integrations.peft because of the following error (look up to see its traceback):
Only a single TORCH_LIBRARY can be used to register the namespace c10d_functional; please put all of your definitions in a single TORCH_LIBRARY block.  If you were trying to specify implementations, consider using TORCH_LIBRARY_IMPL (which can be duplicated).  If you really intended to define operators for a single namespace in a distributed way, you can use TORCH_LIBRARY_FRAGMENT to explicitly indicate this.  Previous registration of TORCH_LIBRARY was registered at /dev/null:488; latest registration was registered at /dev/null:488

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
llm, tokenizer = load_llm("EleutherAI/gpt-neo-2.7B", device = device, cache_dir="D:/Models")

: 

In [ ]:
vision_model , img_proc = load_vision_model("laion/CLIP-ViT-H-14-laion2B-s32B-b79K", device = device, cache_dir="D:/Models")

In [12]:
vision_model.config.hidden_size

1280

In [26]:
torch.cuda.empty_cache()

In [4]:
import torch
from transformers import PreTrainedModel
from loader.model_loader import load_vision_model, load_llm
from vision.projector import load_vision_projector
from vision.feature_select import feature_select
from vision.learned_encoding import load_learned_encoding
from image_handling.padding import resize_with_padding, load_images
from image_handling.slice import split_image
from transformers import BitsAndBytesConfig
import math
import requests
from PIL import Image
from io import BytesIO

class LeMultiModal():
    #Usually they do some config here, but I'm skipping that for now

    def __init__(self):
        super().__init__()
        self.quantization_config = BitsAndBytesConfig(load_in_8bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
        self.vision_model , self.image_processor = load_vision_model("laion/CLIP-ViT-H-14-laion2B-s32B-b79K", cache_dir="D:/Models")
        self.llm, self.tokenizer = load_llm("EleutherAI/gpt-neo-2.7B", cache_dir="D:/Models")
        self.vision_projector = load_vision_projector()
        self.llm_dim = self.llm.config.hidden_size
        self.vision_dim = self.vision_model.config.hidden_size
        self.learned_positional = load_learned_encoding(self.vision_dim, self.llm_dim, "linear")

    def get_positional_encoding(max_seq_len, embedding_dim):
        position_encoding = torch.zeros(max_seq_len, embedding_dim)
        position = torch.arange(0, max_seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * (-math.log(10000.0) / embedding_dim))
        position_encoding[:, 0::2] = torch.sin(position * div_term)
        position_encoding[:, 1::2] = torch.cos(position * div_term)
        return position_encoding

    def prepare(self, inputs, images):
        if(images is None):
            return 0
        images = load_images(images)

    def encode_images_positional_encoding(self, images):
        #make sure all images are already preprocessed 
            image_tensors = self.image_processor.preprocess(images, return_tensors='pt')['pixel_values'].to(self.device)
            with torch.no_grad(): 
                batch_features = self.vison_model(image_tensors, output_hidden_states=True)
                image_features = batch_features.hidden_states[-1]
                image_features = feature_select(image_features, "patch")

                # Positional Encoding
                max_seq_len = image_features.shape[1]
                pos_encoding = self.get_positional_encoding(max_seq_len, image_features.shape[-1]).to(self.device)
                image_features += pos_encoding

            # Learned Encoding
            image_features += self.learned_encoding_layer(image_features)

            return self.projector(image_features)
    
    def images_uhd_positional_encoding(self, image):
        #lower the image with padding to 
        resized_image = resize_with_padding(image, 336)
        splits = split_image(image)
        self.encode_images_positional_encoding(splits)

    @property
    def device(self):
        return "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
model = LeMultiModal()

TypeError: load_vision_projector() missing 2 required positional arguments: 'input' and 'output'

In [2]:
vision_model , image_processor = load_vision_model("laion/CLIP-ViT-H-14-laion2B-s32B-b79K", device = device, cache_dir="D:/Models" )
llm, tokenizer = load_llm("EleutherAI/gpt-neo-2.7B", device = device, cache_dir="D:/Models")
vision_projector = load_vision_projector()
embedding_dim = llm.config.hidden_size
learned_positional = load_learned_encoding(embedding_dim, "linear")

NameError: name 'load_vision_model' is not defined

In [ ]:
def get_positional_encoding(max_seq_len, embedding_dim):
        position_encoding = torch.zeros(max_seq_len, embedding_dim)
        position = torch.arange(0, max_seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * (-math.log(10000.0) / embedding_dim))
        position_encoding[:, 0::2] = torch.sin(position * div_term)
        position_encoding[:, 1::2] = torch.cos(position * div_term)
        return position_encoding